In [1]:
import pandas as pd
import numpy as np

import mysql.connector
from sqlalchemy import create_engine
import pandas.io.sql as psql
import sqlalchemy

from dateutil import parser
from datetime import datetime
from datetime import date
from dateutil.relativedelta import relativedelta


In [32]:
database_username = 'root'
database_password = 'yash'
database_ip       = '127.0.0.1'
database_name     = 'mainschema'
database_connection = sqlalchemy.create_engine('mysql+mysqlconnector://{0}:{1}@{2}/{3}'.
                                               format(database_username, database_password, 
                                                      database_ip, database_name))

In [33]:
df = pd.read_sql('SELECT * FROM marketsmojorecos', con=database_connection)
del df['srno']
del df['index']


In [34]:
df

broker         company_name  current_price  \
0            Prabhudas Lilladher  Colgate Palmol. (I)        1248.25   
1                    Axis Direct      Larsen & Toubro        1310.90   
2                    Nirmal Bang       Britannia Inds        2695.00   
3                 Sushil Finance  Sahyadri Industries         147.00   
4      Geojit Financial Services       Tata Chemicals         580.95   
...                          ...                  ...            ...   
22115        Prabhudas Lilladher                 BPCL         493.00   
22116                Axis Direct        Bajaj Finance        4056.35   
22117  Geojit Financial Services             Hindalco         207.60   
22118  Geojit Financial Services                 HDFC        2368.00   
22119  Geojit Financial Services    Grasim Industries         768.20   

      recomended_buying predict_date target_price  
0               1274.35   2019-09-03      1159.00  
1                1328.1   2019-09-03      1487.00  
2                2704.8   2019-09-03      3220.00  
3                 145.5   2019-09-03       220.00  
4                579.06   2019-08-30       531.00  
...                 ...          ...          ...  
22115            378.64   2019-09-05       466.00  
22116           3243.58   2019-09-04      3591.00  
22117            179.21   2019-09-04       195.00  
22118           2096.68   2019-09-04      2284.00  
22119            694.82   2019-09-04       803.00  

[22120 rows x 6 columns]

In [35]:
df.drop_duplicates(inplace = True)
df

broker          company_name  current_price  \
0            Prabhudas Lilladher   Colgate Palmol. (I)        1248.25   
1                    Axis Direct       Larsen & Toubro        1310.90   
2                    Nirmal Bang        Britannia Inds        2695.00   
3                 Sushil Finance   Sahyadri Industries         147.00   
4      Geojit Financial Services        Tata Chemicals         580.95   
...                          ...                   ...            ...   
21738              Motilal Oswal  Multi Commodity Exch        1122.00   
21739                Axis Direct         Bajaj Finance        4056.35   
21740  Geojit Financial Services              Hindalco         207.60   
21741  Geojit Financial Services                  HDFC        2368.00   
21742  Geojit Financial Services     Grasim Industries         768.20   

      recomended_buying predict_date target_price  
0               1274.35   2019-09-03      1159.00  
1                1328.1   2019-09-03      1487.00  
2                2704.8   2019-09-03      3220.00  
3                 145.5   2019-09-03       220.00  
4                579.06   2019-08-30       531.00  
...                 ...          ...          ...  
21738            864.94   2019-09-05      1000.00  
21739           3243.58   2019-09-04      3591.00  
21740            179.21   2019-09-04       195.00  
21741           2096.68   2019-09-04      2284.00  
21742            694.82   2019-09-04       803.00  

[13935 rows x 6 columns]

In [36]:
start_date = '2016-01-01'
end_date = '2016-12-31'
date_object1 = datetime.strptime(start_date, '%Y-%m-%d').date()
date_object2 = datetime.strptime(end_date, '%Y-%m-%d').date()

In [37]:
mask = (df['predict_date'] > date_object1) & (df['predict_date'] <= date_object2)

# 2016 data

In [38]:
df1 = df.loc[mask]
df1 = df1.reset_index()
#del df1['level_0']
#del df1['srno']
del df1['index']

In [39]:
print("No of Brokers in 2016 :",df1['broker'].nunique()) 
s1=len(df1)

No of Brokers in 2016 : 61


## CLEANING

In [40]:
df3=pd.DataFrame()
for i in range(0,len(df1)): 
    sym=df1.at[i, 'company_name']
    sym = sym.lower()
    sym = sym.replace(" ","")
    sym = sym.replace(".","")
    sym = sym.replace("(i)","")
    sym = sym.replace("(","")
    sym = sym.replace(")","")
    sym = sym.replace("-","")
    sqlrun = "SELECT SYMBOL FROM symbolsmapped WHERE recoName = '"+sym+"'"
    df3 = df3.append(pd.read_sql(sqlrun, con=database_connection),ignore_index=True)

In [41]:
len(df1)

3037

## SYMBOLS ADDED TO TABLE

In [42]:
df4 =pd.concat([df1,df3],axis=1)
df4

broker          company_name  current_price  \
0             KRChoksey          Force Motors        1199.90   
1           AUM Capital   Narayana Hrudayalay         239.00   
2              Religare            ICICI Bank         390.15   
3     Enochventures.com      Parag Milk Foods         137.65   
4             Edelweiss  Kewal Kiran Clothing        1000.00   
...                 ...                   ...            ...   
3032          Edelweiss     Ipca Laboratories         957.25   
3033        Nirmal Bang      Aurobindo Pharma         605.95   
3034    ICICIdirect.com             HCL Tech.        1108.65   
3035      Angel Broking  Dewan Hsg Fin. Corp.          47.15   
3036    ICICIdirect.com         Ultratech Cem        3883.75   

     recomended_buying predict_date target_price      SYMBOL  
0              3497.00   2016-10-03      4820.00    FORCEMOT  
1               325.55   2016-09-27       400.00          NH  
2               217.73   2016-08-04       181.82   ICICIBANK  
3                329.6   2016-07-12       425.00   PARAGMILK  
4              1796.00   2016-05-25      2039.00        KKCL  
...                ...          ...          ...         ...  
3032            558.00   2016-04-07       440.00     IPCALAB  
3033            762.15   2016-04-07       894.00  AUROPHARMA  
3034            840.00   2016-04-06      1000.00     HCLTECH  
3035            189.00   2016-04-05       270.00        DHFL  
3036           3237.00   2016-04-01      3600.00  ULTRACEMCO  

[3037 rows x 7 columns]

In [43]:
uniqueBrokers = df4.broker.unique()
uniqueBrokers

array(['KRChoksey', 'AUM Capital', 'Religare', 'Enochventures.com',
       'Edelweiss', 'Motilal Oswal', 'Kotak Securities', 'Angel Broking',
       'Geojit Financial Services', 'Centrum', 'IIFL', 'ICICIdirect.com',
       'Axis Direct', 'Moneylicious', 'CD Equisearch', 'Nirmal Bang',
       'Arihant Capital Markets', 'Prabhudas Lilladher',
       'Wallfort Financial', 'HDFC Securities', 'BP Wealth',
       'Rudra Shares & Stocks Brokers', 'GEPL Capital', 'Dynamic Levels',
       'Choice Equity Broking', 'Right Horizons',
       'Systematix Shares & Stocks', 'East India Securities Ltd',
       'Sethi Finmart', 'Maximus Securities', 'Ajcon Global',
       'India Nivesh', 'Way2Wealth', 'YES Securities', 'AnandRathi',
       'SPA Securities', 'Khambatta Securities', 'Reliance Securities',
       'Dolat Capital', 'LKP Securities Limited',
       'Cholamandalam Securities', 'JM Financial', 'Ashika Stock Broking',
       'Antique Stock Broking', 'Karvy Stock Broking',
       'Blue Ocean Stra

## PERFORMANCE DEDUCTION ALGORITHM

In [45]:
#del brokerPerformace
brokerPerformace = pd.DataFrame(columns=['Name','Hit Ratio','Avg. Growth','Total Recos'])
brokerPerformace

Empty DataFrame
Columns: [Name, Hit Ratio, Avg. Growth, Total Recos]
Index: []

In [46]:
for brokerName in uniqueBrokers :
    dfbrokerRecos = df4.loc[df4['broker'] == brokerName]
    print('brokerName : ',brokerName)
    hits=0
    growthSum=0
    total=0
    for index, row in dfbrokerRecos.iterrows():
        #print(row)
        sym = row['SYMBOL']
        print(type(sym))

        avgGrowth=0
        targetp=0
        recbuying=0
        maxval=0
        growthperc=0
        
        if type(sym) == str :
            mydate1 = row['predict_date']#prediction date
            mydate2 = mydate1 + relativedelta(years=1)#date 1 year after prediction
            dbyear1 = "nse"+str(mydate1.year)
            dbyear2 = "nse"+str(mydate2.year)
        
            sqlrun1 = "SELECT * FROM "+dbyear1+" WHERE Date >= '"+str(mydate1)+"' AND Symbol = '"+sym+"'"
            dfprices1 = pd.read_sql(sqlrun1, con=database_connection)
            del dfprices1['index']
        
            sqlrun2 = "SELECT * FROM "+dbyear2+" WHERE Date <= '"+str(mydate2)+"' AND Symbol = '"+sym+"'"
            dfprices2 = pd.read_sql(sqlrun2, con=database_connection)
            del dfprices2['index']
            dfprices = pd.concat([dfprices1,dfprices2],ignore_index=True)

            targetp = float(row['target_price'])
        
            print("SYMBOL : ",sym)
            dfhits = dfprices.loc[dfprices['Closeval'] >= targetp]
            
            total = total + 1
            
            if not dfhits.empty :
                print("HIT!")
                hits = hits + 1
                recbuying = float(row['recomended_buying'])
                maxval = dfprices['Closeval'].max()
                growthperc = (maxval - recbuying) *100 / recbuying
                print('Growth : ',growthperc)
                if np.isnan(growthperc):
                    total = total-1
                else:    
                    if(growthperc < 200):
                        growthSum = growthSum + growthperc
                    else:
                        print("----------------ALERT--------------")
                        high = recbuying * 1.2
                        low = recbuying * 0.8
                        recDate = row['predict_date']
                        print("recDate : ",recDate)
                        dferr = dfprices.loc[dfprices['Date'] == recDate]
                        if dferr['Closeval'].iloc[0] >= low and dferr['Closeval'].iloc[0] <= high :
                            growthSum = growthSum + growthperc
                        else:
                            total = total - 1
                            hits = hits - 1
            else:
                print("DIDN'T HIT!")
            del dfprices
            del dfhits
    if total != 0:
        hitRatio = int(hits*100 / total)
        avgGrowth = int(growthSum / total)
        print("Broker : ", brokerName, "HitRatio : ", hitRatio, "Avg. Growth : ", avgGrowth, "Total Recos. : ", total)
        brokerPerformace = brokerPerformace.append({'Name':brokerName,'Hit Ratio':hitRatio,'Avg. Growth':avgGrowth,'Total Recos':total}, ignore_index=True)

brokerName :  KRChoksey
<class 'str'>
SYMBOL :  FORCEMOT
DIDN'T HIT!
<class 'str'>
SYMBOL :  SOLARINDS
HIT!
Growth :  80.4476479514416
<class 'str'>
SYMBOL :  BAJFINANCE
HIT!
Growth :  130.15240328253225
<class 'str'>
SYMBOL :  INFIBEAM
HIT!
Growth :  31.394101876675595
<class 'str'>
SYMBOL :  TV18BRDCST
DIDN'T HIT!
<class 'str'>
SYMBOL :  STRTECH
HIT!
Growth :  212.4468085106383
----------------ALERT--------------
recDate :  2016-12-09
<class 'str'>


C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:58: FutureWarning: Comparing Series of datetimes with 'datetime.date'.  Currently, the
'datetime.date' is coerced to a datetime. In the future pandas will
not coerce, and 'the values will not compare equal to the
'datetime.date'. To retain the current behavior, convert the
'datetime.date' to a datetime with 'pd.Timestamp'.


SYMBOL :  SCHNEIDER
DIDN'T HIT!
<class 'str'>
SYMBOL :  BALKRISIND
HIT!
Growth :  81.16484440706478
<class 'str'>
SYMBOL :  TATAMOTORS
DIDN'T HIT!
<class 'str'>
SYMBOL :  INFIBEAM
HIT!
Growth :  40.69856459330143
<class 'str'>
SYMBOL :  EMMBI
HIT!
Growth :  115.56521739130434
<class 'str'>
SYMBOL :  IGL
HIT!
Growth :  860.6547619047619
----------------ALERT--------------
recDate :  2016-11-18
<class 'str'>
SYMBOL :  PETRONET
HIT!
Growth :  159.74063400576367
<class 'str'>
SYMBOL :  MONTECARLO
HIT!
Growth :  42.074592074592076
<class 'str'>
SYMBOL :  MINDAIND
HIT!
Growth :  205.16176470588235
----------------ALERT--------------
recDate :  2016-11-15
<class 'str'>
SYMBOL :  BANKBARODA
HIT!
Growth :  21.58385093167702
<class 'str'>
SYMBOL :  SBIN
DIDN'T HIT!
<class 'str'>
SYMBOL :  MANAPPURAM
DIDN'T HIT!
<class 'str'>
SYMBOL :  SUNPHARMA
DIDN'T HIT!
<class 'str'>
SYMBOL :  BHARATFORG
HIT!
Growth :  199.96470588235292
<class 'str'>
SYMBOL :  ICICIBANK
HIT!
Growth :  33.988671810868496
<cla

SYMBOL :  ICICIBANK
HIT!
Growth :  49.93340375694668
<class 'str'>
SYMBOL :  ICICIBANK
HIT!
Growth :  49.93340375694668
<class 'str'>
SYMBOL :  RELIANCE
HIT!
Growth :  205.4157103723995
----------------ALERT--------------
recDate :  2016-12-30
<class 'str'>
SYMBOL :  BHARATFIN
HIT!
Growth :  75.33919276388771
<class 'str'>
SYMBOL :  GLENMARK
DIDN'T HIT!
<class 'str'>
SYMBOL :  SUNPHARMA
DIDN'T HIT!
<class 'str'>
SYMBOL :  BHARTIARTL
HIT!
Growth :  63.392991427282325
<class 'str'>
SYMBOL :  DIVISLAB
DIDN'T HIT!
<class 'str'>
SYMBOL :  GSPL
HIT!
Growth :  47.55033557046979
<class 'str'>
SYMBOL :  OIL
HIT!
Growth :  49.81831733904778
<class 'str'>
SYMBOL :  SUNPHARMA
DIDN'T HIT!
<class 'str'>
SYMBOL :  LT
HIT!
Growth :  35.75269828137341
<class 'str'>
SYMBOL :  YESBANK
HIT!
Growth :  730.6897008792471
----------------ALERT--------------
recDate :  2016-11-22
<class 'str'>
SYMBOL :  GUJGASLTD
HIT!
Growth :  83.28105693375156
<class 'str'>
SYMBOL :  PETRONET
HIT!
Growth :  159.1729928686450

SYMBOL :  SADBHAV
DIDN'T HIT!
<class 'str'>
SYMBOL :  MARICO
HIT!
Growth :  25.280681378242356
<class 'str'>
SYMBOL :  AXISBANK
HIT!
Growth :  36.66451751558134
<class 'str'>
SYMBOL :  HDFCBANK
HIT!
Growth :  39.73181818181818
<class 'str'>
SYMBOL :  TCS
HIT!
Growth :  11.228675873273755
<class 'str'>
SYMBOL :  INFY
DIDN'T HIT!
<class 'str'>
SYMBOL :  CEATLTD
HIT!
Growth :  29.994222479000943
<class 'str'>
SYMBOL :  RELIANCE
HIT!
Growth :  205.4157103723995
----------------ALERT--------------
recDate :  2016-12-30
<class 'str'>
SYMBOL :  GLENMARK
DIDN'T HIT!
<class 'str'>
SYMBOL :  SUNPHARMA
DIDN'T HIT!
<class 'str'>
SYMBOL :  BHARTIARTL
HIT!
Growth :  63.392991427282325
<class 'str'>
SYMBOL :  GSPL
HIT!
Growth :  47.55033557046979
<class 'str'>
SYMBOL :  OIL
HIT!
Growth :  49.81831733904778
<class 'str'>
SYMBOL :  SUNPHARMA
DIDN'T HIT!
<class 'str'>
SYMBOL :  LT
HIT!
Growth :  35.75269828137341
<class 'str'>
SYMBOL :  YESBANK
HIT!
Growth :  730.6897008792471
----------------ALERT-----

...

SYMBOL :  ENGINERSIN
HIT!
Growth :  152.1742390163443
<class 'str'>
SYMBOL :  JUBILANT
HIT!
Growth :  134.78679504814306
<class 'str'>
SYMBOL :  TECHM
DIDN'T HIT!
<class 'str'>
SYMBOL :  HEXAWARE
HIT!
Growth :  18.26185101580135
<class 'str'>
SYMBOL :  LT
HIT!
Growth :  20.826081135359495
<class 'str'>
SYMBOL :  ICICIBANK
HIT!
Growth :  44.857117500887455
<class 'str'>
SYMBOL :  EICHERMOT
HIT!
Growth :  41.40204575342343
<class 'str'>
SYMBOL :  LICHSGFIN
HIT!
Growth :  54.17406749555952
<class 'str'>
SYMBOL :  INDUSINDBK
HIT!
Growth :  41.84772849703719
<class 'str'>
SYMBOL :  JUBILANT
HIT!
Growth :  166.64479645077645
<class 'str'>
SYMBOL :  MGL
HIT!
Growth :  144.94061757719717
<class 'str'>
SYMBOL :  HCLTECH
DIDN'T HIT!
<class 'str'>
SYMBOL :  BRITANNIA
HIT!
Growth :  36.614816729340404
<class 'str'>
SYMBOL :  HCLTECH
DIDN'T HIT!
<class 'str'>
SYMBOL :  HDFCBANK
HIT!
Growth :  43.22934232715009
<class 'str'>
SYMBOL :  GLENMARK
DIDN'T HIT!
<class 'str'>
SYMBOL :  BRITANNIA
HIT!
Growt

SYMBOL :  LUPIN
HIT!
Growth :  21.667832167832163
<class 'str'>
SYMBOL :  KRBL
HIT!
Growth :  48.123953098827464
<class 'str'>
SYMBOL :  PRAJIND
HIT!
Growth :  14.545454545454543
<class 'str'>
SYMBOL :  TORNTPOWER
HIT!
Growth :  29.945054945054945
<class 'str'>
SYMBOL :  BIOCON
HIT!
Growth :  472.84345047923324
----------------ALERT--------------
recDate :  2016-05-11
<class 'str'>
SYMBOL :  NOCIL
HIT!
Growth :  107.14285714285714
<class 'str'>
SYMBOL :  WELSPUNIND
HIT!
Growth :  17.936364548830678
Broker :  Dynamic Levels HitRatio :  84 Avg. Growth :  65 Total Recos. :  53
brokerName :  Choice Equity Broking
<class 'str'>
SYMBOL :  HCLTECH
HIT!
Growth :  16.718555417185556
<class 'str'>
SYMBOL :  HCLTECH
HIT!
Growth :  16.718555417185556
Broker :  Choice Equity Broking HitRatio :  100 Avg. Growth :  16 Total Recos. :  2
brokerName :  Right Horizons
<class 'str'>
SYMBOL :  ASHOKLEY
HIT!
Growth :  69.0721649484536
<class 'str'>
SYMBOL :  ICICIBANK
HIT!
Growth :  36.95670414499076
<class

SYMBOL :  EMAMILTD
DIDN'T HIT!
<class 'str'>
SYMBOL :  AJANTPHARM
HIT!
Growth :  30.395913154533847
<class 'str'>
SYMBOL :  JKIL
DIDN'T HIT!
<class 'str'>
SYMBOL :  APLLTD
HIT!
Growth :  25.164533820840948
<class 'str'>
SYMBOL :  KEC
HIT!
Growth :  92.01438848920861
<class 'str'>
SYMBOL :  TALWALKARS
DIDN'T HIT!
<class 'str'>
SYMBOL :  RKFORGE
DIDN'T HIT!
<class 'str'>
SYMBOL :  RSWM
HIT!
Growth :  28.643410852713185
<class 'str'>
SYMBOL :  YESBANK
HIT!
Growth :  737.0881567230632
----------------ALERT--------------
recDate :  2016-11-24
<class 'str'>
SYMBOL :  TVSMOTOR
HIT!
Growth :  102.18842154275961
<class 'str'>
SYMBOL :  AJANTPHARM
DIDN'T HIT!
<class 'str'>
SYMBOL :  AJANTPHARM
HIT!
Growth :  30.395913154533847
<class 'str'>
SYMBOL :  JKIL
DIDN'T HIT!
<class 'str'>
SYMBOL :  KEC
HIT!
Growth :  92.01438848920861
<class 'str'>
SYMBOL :  RSWM
HIT!
Growth :  28.643410852713185
Broker :  AnandRathi HitRatio :  57 Avg. Growth :  35 Total Recos. :  19
brokerName :  SPA Securities
<class

SYMBOL :  INDUSINDBK
HIT!
Growth :  38.728888888888896
<class 'str'>
SYMBOL :  NAVKARCORP
DIDN'T HIT!
<class 'str'>
SYMBOL :  INDIACEM
HIT!
Growth :  112.39065654138227
<class 'str'>
SYMBOL :  NBCC
HIT!
Growth :  54.020234291799774
<class 'str'>
SYMBOL :  PERSISTENT
DIDN'T HIT!
<class 'str'>
SYMBOL :  ITC
HIT!
Growth :  53.88148828663298
<class 'str'>
SYMBOL :  INDIACEM
HIT!
Growth :  111.16528056591149
<class 'str'>
SYMBOL :  HCLTECH
DIDN'T HIT!
<class 'str'>
SYMBOL :  AXISBANK
HIT!
Growth :  37.04741379310344
<class 'str'>
SYMBOL :  ULTRACEMCO
HIT!
Growth :  28.56147155974422
<class 'str'>
SYMBOL :  AUROPHARMA
DIDN'T HIT!
<class 'str'>
SYMBOL :  BAJAJ-AUTO
HIT!
Growth :  15.160344767421584
<class 'str'>
SYMBOL :  GODREJCP
HIT!
Growth :  156.94937541091386
<class 'str'>
SYMBOL :  NESTLEIND
HIT!
Growth :  11.63573391559688
<class 'str'>
SYMBOL :  COALINDIA
DIDN'T HIT!
<class 'str'>
SYMBOL :  BAJAJCORP
DIDN'T HIT!
<class 'str'>
SYMBOL :  AMBUJACEM
HIT!
Growth :  12.989045383411588
<clas

SYMBOL :  ESCORTS
HIT!
Growth :  117.45664739884393
<class 'str'>
SYMBOL :  HCC
HIT!
Growth :  68.92857142857142
<class 'str'>
SYMBOL :  MOTHERSUMI
HIT!
Growth :  130.85256712199717
<class 'str'>
SYMBOL :  LUPIN
DIDN'T HIT!
<class 'str'>
SYMBOL :  ULTRACEMCO
HIT!
Growth :  26.45089443042614
<class 'str'>
SYMBOL :  KNRCON
HIT!
Growth :  624.4594594594595
----------------ALERT--------------
recDate :  2016-06-01
<class 'str'>
SYMBOL :  ULTRACEMCO
HIT!
Growth :  30.334869431643614
<class 'str'>
SYMBOL :  JYOTHYLAB
HIT!
Growth :  25.1
<class 'str'>
SYMBOL :  JBMA
HIT!
Growth :  117.55244755244757
<class 'str'>
SYMBOL :  ESCORTS
HIT!
Growth :  117.45664739884393
<class 'str'>
SYMBOL :  HCC
HIT!
Growth :  68.92857142857142
<class 'str'>
SYMBOL :  MOTHERSUMI
HIT!
Growth :  130.85256712199717
<class 'str'>
SYMBOL :  LUPIN
DIDN'T HIT!
<class 'str'>
SYMBOL :  ULTRACEMCO
HIT!
Growth :  26.45089443042614
<class 'str'>
SYMBOL :  ULTRACEMCO
HIT!
Growth :  30.334869431643614
Broker :  Karvy Stock Bro

In [47]:
brokerPerformace

Name Hit Ratio Avg. Growth Total Recos
0                                 KRChoksey        69          50         117
1                               AUM Capital        77          70           9
2                                  Religare        79          36         222
3                         Enochventures.com        52          34          17
4                                 Edelweiss        72          35         278
5                             Motilal Oswal        69          37         299
6                          Kotak Securities        82          43          75
7                             Angel Broking        54          36          66
8                 Geojit Financial Services        83          42         118
9                                   Centrum        72          40         127
10                                     IIFL        68          45          29
11                          ICICIdirect.com        77          46         428
12                              Axis Direct        79          44         264
13                             Moneylicious        82          62          17
14                            CD Equisearch        77          45          27
15                              Nirmal Bang        67          24          55
16                  Arihant Capital Markets        87          41          54
17                      Prabhudas Lilladher        68          39         183
18                       Wallfort Financial         0           0           2
19                          HDFC Securities        84          54          25
20            Rudra Shares & Stocks Brokers        75          30           4
21                             GEPL Capital        75          31           4
22                           Dynamic Levels        84          65          53
23                    Choice Equity Broking       100          16           2
24                           Right Horizons        80          42          10
25               Systematix Shares & Stocks       100          45           9
26                East India Securities Ltd        80         221          10
27                            Sethi Finmart        50          49           8
28                       Maximus Securities       100          36           2
29                             Ajcon Global         0           0           4
30                             India Nivesh        85          60           7
31                               Way2Wealth        66          28          18
32                           YES Securities        85          29          20
33                               AnandRathi        57          35          19
34                           SPA Securities        65          46          55
35                     Khambatta Securities        66          26          18
36                      Reliance Securities        73          35          87
37                            Dolat Capital        60          32          30
38                   LKP Securities Limited         0           0           4
39                 Cholamandalam Securities         0           0           2
40                             JM Financial        65          33          29
41                     Ashika Stock Broking       100          18           2
42                    Antique Stock Broking       100          13           1
43                      Karvy Stock Broking        85          63          14
44            Blue Ocean Strategic Advisors        50          38           4
45                                Sharekhan        54          16          11
46                     Precision Technicals        75          51           4
47                    Investment Imperative       100          92           1
48      Avinash Gorakshakar - Market Expert       100          11           2
49                           Hem Securities       100          56           1
50                        Powermywealth.com        66        

In [48]:
brokerPerformace.to_sql(con=database_connection, name='brokerperformance2016', if_exists='replace')